In [1]:
import pandas as pd
import json
import csv
import ast

In [53]:
df_review = pd.read_csv('review_organizado.csv')
df_user = pd.read_csv('user_items_limpio.csv')
df_steam = pd.read_csv('modified_games.csv')

In [43]:
df_user

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5094100,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5094101,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5094102,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5094103,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [72]:
df_steam.tail()

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,id,metascore,developer,Año_estreno,genre
71549,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,2018-01-04,"['Indie', 'Simulation', 'Racing']",1.39,"['Single-player', 'Steam Achievements', 'Steam...",1.99,610660.0,NaN,Laush Dmitriy Sergeevich,2018,Indie
71550,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,2018-01-04,"['Indie', 'Simulation', 'Racing']",1.39,"['Single-player', 'Steam Achievements', 'Steam...",1.99,610660.0,NaN,Laush Dmitriy Sergeevich,2018,Racing
71551,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,2018-01-04,"['Indie', 'Simulation', 'Racing']",1.39,"['Single-player', 'Steam Achievements', 'Steam...",1.99,610660.0,NaN,Laush Dmitriy Sergeevich,2018,Simulation
71552,SIXNAILS,"['Casual', 'Indie']",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",NaN,"['Single-player', 'Steam Achievements', 'Steam...",4.99,658870.0,NaN,"xropi,stev3ns",2017,Casual
71553,SIXNAILS,"['Casual', 'Indie']",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",NaN,"['Single-player', 'Steam Achievements', 'Steam...",4.99,658870.0,NaN,"xropi,stev3ns",2017,Indie


In [57]:
df_steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71554 entries, 0 to 71553
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       59093 non-null  object 
 1   genres          71554 non-null  object 
 2   app_name        71552 non-null  object 
 3   title           71552 non-null  object 
 4   release_date    71011 non-null  object 
 5   tags            71508 non-null  object 
 6   discount_price  580 non-null    float64
 7   specs           70997 non-null  object 
 8   price           68355 non-null  object 
 9   id              71552 non-null  float64
 10  metascore       5368 non-null   float64
 11  developer       71204 non-null  object 
 12  Año_estreno     71554 non-null  object 
 13  genre           71554 non-null  object 
dtypes: float64(3), object(11)
memory usage: 7.6+ MB


In [58]:
df_steam['release_date'] = pd.to_datetime(df_steam['release_date'], errors='coerce')

In [60]:
def developer(df, developer):
    # Filter data for the specified developer
    df_dev = df_steam[df_steam['developer'] == developer]
    # Get a list of unique years
    years = df_steam['release_date'].dt.year.unique()
    # Iterate over the years
    percentages = []
    for year in years:
        # Filter data for the year
        df_year = df_dev[df_dev['release_date'].dt.year == year]
        # Convert 'price' column to string
        df_year['price'] = df_year['price'].astype(str)
        # Count the number of free contents
        num_free = df_year[df_year['price'].str.lower(
        ).str.contains('free')].shape[0]
        # Calculate the percentage
        if num_free == 0:
            percentage = 0
        else:
            percentage = (num_free / df_year.shape[0]) * 100
        # Add the percentage to the list
        percentages.append(f'{year}: {percentage:.2f}%')
    return percentages


El código anterior define una función llamada "get_free_content_percentage_by_year" que toma un dataframe y el nombre de un desarrollador como entradas. Su objetivo es calcular el porcentaje de contenido gratuito lanzado por el desarrollador especificado para cada año único en el dataframe.

La función primero filtra el dataframe para incluir solo las filas donde el desarrollador coincide con el nombre de desarrollador ingresado. Luego, extrae una lista de años únicos de la columna "release_date" del dataframe filtrado.

A continuación, la función itera sobre cada año en la lista de años. Para cada año, filtra nuevamente el dataframe para incluir solo las filas donde el año de lanzamiento coincide con el año actual que se está iterando.

Luego, cuenta el número de contenidos gratuitos verificando la columna "price" en busca del valor "free to play" en el dataframe filtrado. La función calcula el porcentaje dividiendo el número de contenidos gratuitos por el número total de contenidos para ese año y multiplicándolo por 100.

Si no hay contenidos gratuitos para un año en particular, el porcentaje se establece en 0. Finalmente, la función agrega el año y el porcentaje como una cadena formateada a la lista "percentages".

La función devuelve la lista de porcentajes para cada año.

In [61]:
percentages = developer(df_steam, 'KUMA GAMES')
print(percentages)

['2018.0: 100.00%', '2017.0: 0.00%', 'nan: 0.00%', '1997.0: 0.00%', '1998.0: 0.00%', '2016.0: 0.00%', '2006.0: 0.00%', '2005.0: 0.00%', '2003.0: 0.00%', '2007.0: 0.00%', '2002.0: 0.00%', '2000.0: 0.00%', '1995.0: 0.00%', '1996.0: 0.00%', '1994.0: 0.00%', '2001.0: 0.00%', '1993.0: 0.00%', '2004.0: 0.00%', '1999.0: 0.00%', '2008.0: 0.00%', '2009.0: 0.00%', '1992.0: 0.00%', '1989.0: 0.00%', '2010.0: 0.00%', '2011.0: 0.00%', '2013.0: 0.00%', '2012.0: 0.00%', '2014.0: 0.00%', '1983.0: 0.00%', '1984.0: 0.00%', '2015.0: 0.00%', '1990.0: 0.00%', '1988.0: 0.00%', '1991.0: 0.00%', '1987.0: 0.00%', '1986.0: 0.00%', '2021.0: 0.00%', '2019.0: 0.00%', '1985.0: 0.00%']


In [62]:
items = df_user[["user_id", "item_id", "items_count"]]
items

,user_id,item_id,items_count
0,76561197970982479,10,277
1,76561197970982479,20,277
2,76561197970982479,30,277
3,76561197970982479,40,277
4,76561197970982479,50,277
...,...,...,...
5094100,76561198329548331,346330,7
5094101,76561198329548331,373330,7
5094102,76561198329548331,388490,7
5094103,76561198329548331,521570,7


In [63]:
recommend = df_review[["user_id", "recommend"]]
recommend

,user_id,recommend
0,76561197970982479,"True, True, True"
1,js41637,"True, True, True"
2,evcentric,"True, True, True, True, True, True"
3,doctr,"True, True, True, True, True, True"
4,maplemage,"True, True, True, True"
...,...,...
25794,76561198306599751,True
25795,Ghoustik,True
25796,76561198310819422,True
25797,76561198312638244,"True, True, True, True"


In [64]:
df_steam["price"].isnull().sum()

3199

In [65]:
cant_dinero = df_steam[["price", "id"]]
cant_dinero = cant_dinero.drop_duplicates("id", keep="first")
cant_dinero = cant_dinero.rename(columns={"id": "item_id"})
cant_dinero


,price,item_id
0,4.99,761140.0
5,Free To Play,643980.0
9,Free to Play,670290.0
14,0.99,767400.0
17,3.99,772540.0
...,...,...
71538,1.99,745400.0
71542,1.99,773640.0
71546,4.99,733530.0
71549,1.99,610660.0


In [66]:
cant_items = items.merge(cant_dinero, on="item_id", how="left")
cant_items

,user_id,item_id,items_count,price
0,76561197970982479,10,277,9.99
1,76561197970982479,20,277,4.99
2,76561197970982479,30,277,4.99
3,76561197970982479,40,277,4.99
4,76561197970982479,50,277,4.99
...,...,...,...,...
5094100,76561198329548331,346330,7,Free
5094101,76561198329548331,373330,7,NaN
5094102,76561198329548331,388490,7,Free to Play
5094103,76561198329548331,521570,7,Free To Play


In [67]:
cant_items["price"].isnull().sum()

1018326

In [68]:
def cambio_a_float(valor):

    if pd.isna(valor):  # esta parte de la función es para el caso de que no exista ningun string o float en el campo
        return 0.0
    try:
        flotante = float(valor)  # si el valor es float lo conservo
        return flotante
    except (ValueError, TypeError):  # si el valor es diferente retorno 0.0
        return 0.0


# Aplico esta función a la columna precio
cant_items["price"] = cant_items["price"].apply(cambio_a_float)


In [69]:
def userdata(user_id):
    # Filtrar el dataframe por el user_id especificado
    user_data = cant_items[cant_items["user_id"] == user_id]

    # Obtener la cantidad de items y la suma de precios
    item_count = len(user_data)
    total_price = user_data["price"].sum()

    return {
        "Cantidad de items": item_count,
        "Suma de precios": total_price
    }


Esta función toma un user_id como entrada y filtra un dataframe llamado "cant_items" para obtener los datos específicos del user_id. Luego calcula la cantidad de items y la suma de precios para ese usuario. La función devuelve un diccionario con la cantidad de items y la suma de precios como valores.

In [70]:
user_id = "EchoXSilence"
userdata(user_id)

{'Cantidad de items': 23, 'Suma de precios': 189.84}

In [73]:
# Se extraen las columnas para generar un nuevo dataframe y correr la función
genre = df_steam[["genre", "id", 'title']]
genre


,genre,id,title
0,Action,761140.0,Lost Summoner Kitty
1,Casual,761140.0,Lost Summoner Kitty
2,Indie,761140.0,Lost Summoner Kitty
3,Simulation,761140.0,Lost Summoner Kitty
4,Strategy,761140.0,Lost Summoner Kitty
...,...,...,...
71549,Indie,610660.0,Russian Roads
71550,Racing,610660.0,Russian Roads
71551,Simulation,610660.0,Russian Roads
71552,Casual,658870.0,EXIT 2 - Directions


In [75]:
# Se extraen las columnas para generar un nuevo dataframe y correr la función
genre = df_steam[["genre", "id"]]
# Cambiamos "id" a "item_id" para poder utilizar el merge
genre = genre.rename(columns={"id": "item_id"})
genre


,genre,item_id
0,Action,761140.0
1,Casual,761140.0
2,Indie,761140.0
3,Simulation,761140.0
4,Strategy,761140.0
...,...,...
71549,Indie,610660.0
71550,Racing,610660.0
71551,Simulation,610660.0
71552,Casual,658870.0


In [74]:
tiempo = df_user[["item_id", "playtime_forever"]]
tiempo


,item_id,playtime_forever
0,10,6
1,20,0
2,30,7
3,40,0
4,50,0
...,...,...
5094100,346330,0
5094101,373330,0
5094102,388490,3
5094103,521570,4


In [76]:
genero = tiempo.merge(genre, on="item_id")
genero


,item_id,playtime_forever,genre
0,10,6,Action
1,10,0,Action
2,10,0,Action
3,10,93,Action
4,10,108,Action
...,...,...,...
9877299,354280,164,Indie
9877300,354280,164,Simulation
9877301,433920,0,Adventure
9877302,433920,0,Indie


In [77]:
# agrupo los generos por la columna "playtime_forever" y reseteo el indice
rank_genre = genero.groupby("genre")["playtime_forever"].sum().reset_index()
# ordeno los valores
rank_genre = rank_genre.sort_values(by="playtime_forever", ascending=False)
# crear la columna de ranking
# se empieza a contar desde uno
rank_genre['ranking'] = range(1, len(rank_genre)+1)
rank_genre


,genre,playtime_forever,ranking
0,Action,3075100464,1
9,Indie,1475473529,2
12,RPG,1027927921,3
1,Adventure,898763639,4
14,Simulation,855263068,5
17,Strategy,650996822,6
8,Free to Play,603643302,7
10,Massively Multiplayer,441125566,8
4,Casual,249316634,9
6,Early Access,156692536,10


In [81]:
# La siguiente función retorna el ranking en que se ubica el genero que se le ingresa de acuerdo a "playtime_forever"
"""
Parametros:
        - genero (str): el genero de juegos Steam que se quiera conocer el tiempo jugado
Retorna:
        - orden: La ubicación dentro de ranking de acuerdo a la columna "playtime_forever"
"""


def genre(genero):
    # filtro el dataframe "rank_genre" para quesu columna "genres" sea igual a el dato que se ingresa
    # a partir de esto se selecciona la columna "ranking" del conjunto resultante y se bloquea para obtener el valor
    orden = rank_genre[rank_genre["genre"] == genero]["ranking"].iloc[0]
    return {
        "El género": genero,
        "se encuentra en el puesto": orden
    }


In [82]:
genre("Simulation")


{'El género': 'Simulation', 'se encuentra en el puesto': 5}

In [83]:
genre = df_steam[["genre", "id"]]
# Cambiamos "id" a "item_id" para poder utilizar el merge
genre = genre.rename(columns={"id": "item_id"})
genre


,genre,item_id
0,Action,761140.0
1,Casual,761140.0
2,Indie,761140.0
3,Simulation,761140.0
4,Strategy,761140.0
...,...,...
71549,Indie,610660.0
71550,Racing,610660.0
71551,Simulation,610660.0
71552,Casual,658870.0


In [84]:
play = df_user[["item_id", "playtime_forever", "user_id", "user_url"]]
play


,item_id,playtime_forever,user_id,user_url
0,10,6,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,7,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...
5094100,346330,0,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5094101,373330,0,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5094102,388490,3,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5094103,521570,4,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [85]:
play = play.merge(genre, on="item_id")
play


,item_id,playtime_forever,user_id,user_url,genre
0,10,6,76561197970982479,http://steamcommunity.com/profiles/76561197970...,Action
1,10,0,js41637,http://steamcommunity.com/id/js41637,Action
2,10,0,Riot-Punch,http://steamcommunity.com/id/Riot-Punch,Action
3,10,93,doctr,http://steamcommunity.com/id/doctr,Action
4,10,108,corrupted_soul,http://steamcommunity.com/id/corrupted_soul,Action
...,...,...,...,...,...
9877299,354280,164,76561198107283457,http://steamcommunity.com/profiles/76561198107...,Indie
9877300,354280,164,76561198107283457,http://steamcommunity.com/profiles/76561198107...,Simulation
9877301,433920,0,inven,http://steamcommunity.com/id/inven,Adventure
9877302,433920,0,inven,http://steamcommunity.com/id/inven,Indie


In [86]:
play = play.drop("item_id", axis=1)
play

,playtime_forever,user_id,user_url,genre
0,6,76561197970982479,http://steamcommunity.com/profiles/76561197970...,Action
1,0,js41637,http://steamcommunity.com/id/js41637,Action
2,0,Riot-Punch,http://steamcommunity.com/id/Riot-Punch,Action
3,93,doctr,http://steamcommunity.com/id/doctr,Action
4,108,corrupted_soul,http://steamcommunity.com/id/corrupted_soul,Action
...,...,...,...,...
9877299,164,76561198107283457,http://steamcommunity.com/profiles/76561198107...,Indie
9877300,164,76561198107283457,http://steamcommunity.com/profiles/76561198107...,Simulation
9877301,0,inven,http://steamcommunity.com/id/inven,Adventure
9877302,0,inven,http://steamcommunity.com/id/inven,Indie


In [87]:
# La siguiente función retorna el TOP 5 de usuarios junto a su información, con mayor horas de juego en el genero que se le indica
"""
Parametros:
        -genero (str): El genero de juego Steam del que se necesita conocer el TOP 5 de usuarios
Retorna:
        -top_5_users(list) : Lista ordenada por horas de juego "playtime_forever", en forma descendente, conteniendo los nombres de usuario y dirección url
"""


def userforgenre(genero):
    # se filtra el dataframe con la columna "genres" y se la iguala con el dato ingresado
    genre_data = play[play["genre"] == genero]
    # agrupo el genero por usuario y horas jugadas
    user_hours = genre_data.groupby(["user_id", "user_url"])[
        "playtime_forever"].sum().reset_index()
    # ordena los valores de mayor a menor
    user_hours = user_hours.sort_values(by="playtime_forever", ascending=False)
    # extrae los primeros 5
    top_5_users = user_hours.head(5)
    return top_5_users


In [88]:
genero = "Action"
userforgenre(genero)


,user_id,user_url,playtime_forever
50657,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699307
64497,shinomegami,http://steamcommunity.com/id/shinomegami,1580428
49457,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212
51237,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,1065742
44133,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,1061193
